In [11]:
contest_folder = 'round1'

In [12]:
from analysis import get_current_status, select_solution

In [13]:
df_grouped, aggregated_df = get_current_status(contest_folder)

In [14]:
aggregated_list = aggregated_df.to_dict(orient='records')

In [15]:
aggregated_df

,problem_code,problem_name,statement,sample_in,sample_out,full_in,solution_id,execution_response,execution_code,execution_sample_out,execution_full_out,hash,status,timestring
0,substitution_cipher,Substitution Cipher,You’ve probably seen the most classic substitu...,6\n??2 3\n135201 1\n?35 2\n1?0 2\n1122 1\n3???...,Case #1: 122 3\nCase #2: 135201 2\nCase #3: 13...,128\n??2 3\n135201 1\n?35 2\n1?0 2\n1122 1\n3?...,"[004, 006, 007, 010, 012, 014, 015, 017, 018]","[To solve this problem efficiently, several ke...",[import sys\nimport sys\nimport sys\nimport sy...,[Case #1: 972 1\nCase #2: 135201 1\nCase #3: 8...,[Case #1: 972 1\nCase #2: 135201 1\nCase #3: 8...,31c563a5,not_analyzed,0612


In [28]:
import time
import threading

while True:
    _, aggregated_df = get_current_status(contest_folder)
    if len(aggregated_df) == 0:
        print("Not outputs found yet")
        time.sleep(30)
        continue
    
    aggregated_list = aggregated_df.to_dict(orient='records')
    print(f"Running analysis at {aggregated_df['timestring'][0]}")
    
    for row in aggregated_list:
        if row["status"] != "not_analyzed":
            continue
        analysis_thread = threading.Thread(target=select_solution, args=(row,))
        analysis_thread.start()
        print(f"Running analysis at {row['timestring']} for {row['problem_name']}")

    time.sleep(30)

Running analysis at 1956
Running analysis at 1956 for Prime Subtractorization
Running analysis at 1956 for Subsonic Subway
Running analysis at 1956 for Substantial Losses
Running analysis at 1956 for Substitution Cipher
Running analysis at 1956 for Wildcard Submissions


Exception in thread Thread-69:
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/htong/Desktop/hackercup/analysis.py", line 274, in process_row
    )
  File "/Users/htong/Desktop/hackercup/analysis.py", line 213, in call_openai
    
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/openai/_utils/_utils.py", line 272, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/openai/resources/chat/completions.py", line 645, in create
    return self._post(
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/openai/_base_client.py", line 1088, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
  

Running analysis at 2026
Running analysis at 2056
Running analysis at 2126
Running analysis at 2156
Running analysis at 2226
Running analysis at 2256
Running analysis at 2326


KeyboardInterrupt: 

In [8]:
process_row(aggregated_list[3])

("After carefully reviewing all the presented solutions, here's the analysis:\n\n### **Solution Analysis**\n\n1. **Solution ID 000**\n   - **Issue:** The output for the first test case (`Case #1: 112 116`) does not match the expected sample output (`Case #1: 122 3`). This indicates a flaw in both the decoding logic and the way corrupted characters are handled.\n   \n2. **Solution ID 001**\n   - **Strengths:**\n     - **Correctness:** The output matches the sample input perfectly. For example, `Case #1: 122 3` aligns with the expected output.\n     - **Comprehensive Handling:** The solution effectively uses dynamic programming to maximize decodings and correctly identifies the lexicographically \\(K\\)th largest string.\n     - **Efficiency:** It accounts for large inputs by implementing optimizations and handles edge cases gracefully.\n   - **Conclusion:** This solution accurately solves the problem as per the given specifications and sample tests.\n\n3. **Solution ID 003**\n   - **Iss